In [3]:
import pandas as pd

lol = [[1,2],[1,2]]
df = pd.DataFrame(lol, columns=['a','b'])
df

,a,b
0,1,2
1,1,2


In [5]:
df.to_parquet('my_file.parquet', engine='pyarrow')

In [15]:
import requests
r = requests.get("https://github.com/Phil-Grim/london_properties_analysis/blob/main/dtypes.yaml")
r.text

from yaml import safe_load

with open(r.text, "rb") as schema_yaml:
    schema = safe_load(schema_yaml)["raw_dtypes"]



OSError: [Errno 36] File name too long: '{"payload":{"allShortcutsEnabled":false,"fileTree":{"":{"items":[{"name":"flows","path":"flows","contentType":"directory"},{"name":"terraform","path":"terraform","contentType":"directory"},{"name":".gitignore","path":".gitignore","contentType":"file"},{"name":"README.md","path":"README.md","contentType":"file"},{"name":"dtypes.yaml","path":"dtypes.yaml","contentType":"file"},{"name":"managed-execution.py","path":"managed-execution.py","contentType":"file"},{"name":"requirements.txt","path":"requirements.txt","contentType":"file"},{"name":"testing.ipynb","path":"testing.ipynb","contentType":"file"}],"totalCount":8}},"fileTreeProcessingTime":2.3304419999999997,"foldersToFetch":[],"repo":{"id":756258139,"defaultBranch":"main","name":"london_properties_analysis","ownerLogin":"Phil-Grim","currentUserCanPush":false,"isFork":false,"isEmpty":false,"createdAt":"2024-02-12T09:58:24.000Z","ownerAvatar":"https://avatars.githubusercontent.com/u/140601461?v=4